Imports

In [ ]:
from getpass import getpass

# Enter your GitHub token securely
token = getpass('Enter your GitHub token: ')

# Clone the repo using the token
!git clone https://{token}@github.com/ellen660/Emojiville.git

In [ ]:
# Navigate into the repo folder
%cd /content/Emojiville

!git pull

# Import necessary modules or files
import sys
sys.path.append('/content/Emojiville')

In [4]:
# load in class from gpt.py file in same directory
#pip install openai, pandas and torch

!pip install transformers
!pip install torch
!pip install pandas
!pip install openai
!pip install tiktoken
!pip install tensorboard
!pip install torchmetrics


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/ed/ad/c9b96572ab7994e73c64588f8875741823f2daba70e746547fff9a2d9a54/transformers-4.46.2-py3-none-any.whl.metadata
  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.46.2-py3-none-any.whl (10.0 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\zhang\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\zhang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\zhang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\zhang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\zhang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\zhang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\zhang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Obtaining dependency information for torchmetrics from https://files.pythonhosted.org/packages/6d/5f/61483e589f87c3709236ce6ee5d44ffeadec12d04b1b0ae3e0900667edf5/torchmetrics-1.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for lightning-utilities>=0.8.0 from https://files.pythonhosted.org/packages/2e/5d/4c4672025c23a305231a81bf492f65aa3ea0965a89f9ca369a9ee7d47fd9/lightning_utilities-0.11.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/891.4 kB ? eta -:--:--
   --- ------------------------------------ 71.7/891.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------- ----------- 645.1/891.4 kB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 891.4/891.4 kB 9.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\zhang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
from gpt import SentimentDataset
from linear_probe import LinearProbe
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from metrics import MulticlassMetrics
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device, {device}")

device, cpu


Dataset

In [ ]:
torch.manual_seed(42) #making it random but reproducible
csv_file = 'cleaned_backhand_index_pointing_right_with_sentiment.csv'
batch_size = 48

dataset = SentimentDataset(csv_file, debug=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for each subset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Evaluation Metrics

In [9]:
metrics = MulticlassMetrics(device=device, num_classes=3)

writer = SummaryWriter(log_dir=f"/content/logs")

#Logger for tensorboard
def logger(writer, metrics, phase, epoch_index):

    for key, value in metrics.items():
        # if key == 'confusion':
        #     fig = plot_confusion_matrix(metrics[key])
        #     writer.add_figure(f"Confusion Matrix {phase}", fig, epoch_index)
        # else:
        ## check for 2 class multiclass
        if type(value)!= float and len(value.shape) > 0 and value.shape[0] == 2:
            value = value[1]
        elif type(value)!= float and len(value.shape) > 0 and value.shape[0] > 2:
            #average 
            value = torch.nanmean(value)
            # breakpoint()

            # raise Exception("Need to handle multiclass")
            # bp()
        writer.add_scalar("%s/%s"%(phase, key), value, epoch_index)
    writer.flush()



Model 

In [10]:
#script to train the model

embedding_dim = 1536
num_classes = 3 #sentiment classes -1, 0, 1
num_epochs = 10
learning_rate = 2e-3
weight_decay =0.00001

model = LinearProbe(embedding_dim, num_classes)
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay = weight_decay)

Training

In [ ]:
for epoch in tqdm(range(num_epochs), desc="Epochs", unit="epoch"):
    #Validation
    model.eval()
    with torch.no_grad():
        epoch_loss = 0
        for i, (x, y) in enumerate(tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}", unit="batch")):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)

            breakpoint()
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()
            metrics.fill_metrics(y_pred, y)
        print(f"Epoch {epoch}, validation loss: {epoch_loss}")
        computed_metrics = metrics.compute_and_log_metrics(epoch_loss)
        logger(writer, computed_metrics, 'val', epoch)
        metrics.clear_metrics()
    #Training
    model.train()
    epoch_loss = 0
    for i, (x, y) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch + 1}", unit="batch")):
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        metrics.fill_metrics(y_pred, y)
    print(f"Epoch {epoch}, training loss: {epoch_loss}")
    computed_metrics = metrics.compute_and_log_metrics(epoch_loss)
    logger(writer, computed_metrics, 'train', epoch)
    metrics.clear_metrics()

# Save the final model at the end of all epochs
model_save_path = f"/content/model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Final model saved at {model_save_path}")
writer.close()

Epochs:   0%|          | 0/10 [04:23<?, ?epoch/s]


KeyboardInterrupt: 